# Simulation Truth

This notebook will introduce you to the concept of simulation truth in fuse.



## Imports and Simulation Context

Similar to the previous notebooks, we will start by importing the necessary modules and creating a simulation context.

In [ ]:
import fuse
import numpy as np

In [ ]:
st = fuse.context.full_chain_context(output_folder="./fuse_data")

st.set_config(
    {
        "path": "/project2/lgrandi/xenonnt/simulations/testing",
        "file_name": "pmt_neutrons_100.root",
        "entry_stop": 10,
    }
)

run_number = "00000"

## Raw_Records and Contributing_Clusters

First we will run the simulation up to `raw_records`.

In [ ]:
st.make(run_number, "microphysics_summary")
st.make(run_number, "photon_summary")
st.make(run_number, "raw_records")

Now that the main simulation output is produced, we can build some truth information. First we will build `records_truth`. 

In [ ]:
st.make(run_number, "records_truth")

Both `raw_records`and `records_truth` are of the same data_kind and can be loaded together. 

In [ ]:
raw_records = st.get_array(run_number, ["raw_records", "records_truth"])

`contributing_clusters` gives you five additional columns. These are:
- `s1_photons_in_record` - The number of S1 photons in the `raw_record`
- `s2_photons_in_record` - The number of S2 photons in the `raw_record`
- `ap_photons_in_record` - The number of (virtual) PMT afterpulse 'photons' in the record
- `raw_area` - The sum of the contributing photon gains divided by the gain of the PMT

Lets have a look on the number of photons that make it into the first record: 

In [ ]:
index = 0
print("S1 photons:", raw_records[index]["s1_photons_in_record"])
print("S2 photons:", raw_records[index]["s2_photons_in_record"])
print("AP photons:", raw_records[index]["ap_photons_in_record"])
print("Raw area:", raw_records[index]["raw_area"])

## Peaks and peak_truth

Next we can process the simulation result to `peak_basics`. Strax(en) will merge multiple records into a peak. The PeakTruth plugin will evaluate which photons contribute to a peak and calculate a truth output for each peak. The provided columns for each peak are:
- `s1_photons_in_peak` - The number of S1 photons that contributed to the peak
- `s2_photons_in_peak` - The number of S2 photons that contributed to the peak
- `ap_photons_in_peak` - The number of (virtual) PMT afterpulse 'photons' that contributed to the peak
- `raw_area_truth` - The sum of all contributing photon gains divided by the gains of the PMTs
- `observable_energy_truth` - Estimate of the energy that is associated with the peak.
- `number_of_contributing_clusters_s1` - Number of clusters that contributed to the peak with S1 photons
- `number_of_contributing_clusters_s2` - Number of clusters that contributed to the peak with S2 photons
- `average_x_of_contributing_clusters` - Weighted average of the x position of the clusters that contributed to the peak
- `average_y_of_contributing_clusters` - Weighted average of the y position of the clusters that contributed to the peak
- `average_z_of_contributing_clusters` - Weighted average of the z position of the clusters that contributed to the peak
- `average_x_obs_of_contributing_clusters` - Weighted average of the observed x position of the clusters that contributed to the peak
- `average_y_obs_of_contributing_clusters` - Weighted average of the observed y position of the clusters that contributed to the peak
- `average_z_obs_of_contributing_clusters` - Weighted average of the observed z position of the clusters that contributed to the peak

Lets take a closer look at `observable_energy_truth` using an example: 
If we would have two clusters, the first one with 100 keV energy producig 100 S1 photons and the second one with 10 keV producing 10 S1 photons. After simulation and processing we find two S1 peaks in our data. The first S1 consitis of 90 photons from the first cluster and 5 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 90/100 * 100 keV + 5/10 * 10 keV = 90 keV + 5 keV = 95 keV. The second S1 consists of 3 photons from the first cluster and 4 photons of the second cluster. The `observable_energy_truth` for this peak is calculated as: 3/100 * 100 keV + 4/10 * 10 keV = 3 keV + 4 keV = 7 keV. A similar calculation is done for the S2 peaks but replacing the S1 photons with the S2 photons.


In [ ]:
st.make(run_number, "peak_positions")
st.make(run_number, "peak_truth")

As strax(en) will take care of the matching of our truth information to the individual peaks, we can simply load the `peak_basics` and `peak_truth` data together.

In [ ]:
peak_basics = st.get_df(run_number, ["peak_basics", "peak_truth", "peak_positions"])

For a peak area bias study we could now compare the raw_area to the peak area:

In [ ]:
peak_basics[["area", "raw_area_truth"]].head()

We might also be interested in the peak classification: 

In [ ]:
peak_basics[["type", "s1_photons_in_peak", "s2_photons_in_peak", "ap_photons_in_peak"]].head()

Or you might want to check how our position reconstruction is doing: 

In [ ]:
peak_basics[
    [
        "type",
        "x",
        "y",
        "average_x_obs_of_contributing_clusters",
        "average_y_obs_of_contributing_clusters",
        "average_z_obs_of_contributing_clusters",
    ]
].head()

## Surviving Clusters
Next lets evaluate if an energy deposit makes it into a peak. This is done by the `SurvivingClusters` plugin. It will provide the following columns:
- `creating_a_photon` - Boolean if the cluster created a propagated photon
- `in_a_peak` - Boolean if the cluster is in a peak

In [ ]:
st.make(run_number, "surviving_clusters")
microphysics_summary = st.get_df(run_number, ["microphysics_summary", "surviving_clusters"])

Now that we have the data loaded we could have a look at clusters that did not make it into a peak: 

In [ ]:
microphysics_summary.query("in_a_peak == False").head()

## Event Truth

Lets move from the peak level to event level data. This is done by the `EventTruth` plugin. It will provide the following columns:
- `x_obs_truth` - The x position of the event. This corresponds to the x position of the main S2.
- `y_obs_truth` - The y position of the event. This corresponds to the y position of the main S2.
- `z_obs_truth` - The z position of the event. This is calculated as mean of the main S1 and S2 `average_z_obs_of_contributing_clusters`. Does this make sense?
- `energy_of_main_peaks_truth` - This is intended to be the energy that can be found in the main S1 and S2. It is calculated as the mean of the `observable_energy_truth` of the main S1 and S2. Does this make any sense???
- `total_energy_in_event_truth` - The sum of all energy deposits that are in the event

In [ ]:
st.make(run_number, "event_truth")

In [ ]:
event_data = st.get_df(run_number, ["event_info", "event_truth"])

First lets take a look at the energy informations: 

In [ ]:
event_data[["e_ces", "energy_of_main_peaks_truth", "total_energy_in_event_truth"]]

And the positions: 

In [ ]:
event_data[["x", "x_obs_truth", "z", "z_obs_truth"]]

## Cluster Tagging

Finally we can investigate if a cluster contributed to the main or alternative S1 or S2. This is done by the `ClusterTagging` plugin. It will provide the following columns:
- `in_main_s1` - Boolean if the cluster contributed to the main S1
- `in_main_s2` - Boolean if the cluster contributed to the main S2
- `in_alt_s1` - Boolean if the cluster contributed to an alternative S1
- `in_alt_s2` - Boolean if the cluster contributed to an alternative S2
- `photons_in_main_s1` - Number of photons the cluster contributed to the main S1
- `photons_in_main_s2` - Number of photons the cluster contributed to the main S2
- `photons_in_alt_s1` - Number of photons the cluster contributed to the alternative S1
- `photons_in_alt_s2` - Number of photons the cluster contributed to the alternative S2

In [ ]:
st.make(run_number, "tagged_clusters")

We can load it together with e.g. microphysics_summary:

In [ ]:
ms_with_tagged_clusters = st.get_df(
    run_number, ["microphysics_summary", "tagged_clusters", "s2_photons_sum"]
)

Lets take a look at some cluster information that contributed to the main S2 of the second event: 

In [ ]:
ms_with_tagged_clusters_cut = ms_with_tagged_clusters.query("eventid == 1 & in_main_s2 == True")
ms_with_tagged_clusters_cut[["ed", "sum_s2_photons", "photons_in_main_s2"]].head(10)